In [1]:
%matplotlib inline

# Preprocessing and Spike Sorting Tutorial

- In this introductory example, you will see how to use the :code:`spikeinterface` to perform a full electrophysiology analysis.
- We will first create some simulated data, and we will then perform some pre-processing, run a couple of spike sorting algorithms, inspect and validate the results, export to Phy, and compare spike sorters.


In [2]:
import os
import pickle
import _pickle as cPickle
import glob
import warnings
import imp
import shutil

/tmp/ipykernel_291467/868811323.py:6: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp


In [3]:
from collections import defaultdict
import time
import json
from datetime import datetime

In [4]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
import numpy as np
import pandas as pd
import scipy.signal

In [5]:
# Changing the figure size
from matplotlib.pyplot import figure
figure(figsize=(8, 6), dpi=80)

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

The spikeinterface module by itself import only the spikeinterface.core submodule
which is not useful for end user



In [6]:
import spikeinterface

We need to import one by one different submodules separately (preferred).
There are 5 modules:

- :code:`extractors` : file IO
- :code:`toolkit` : processing toolkit for pre-, post-processing, validation, and automatic curation
- :code:`sorters` : Python wrappers of spike sorters
- :code:`comparison` : comparison of spike sorting output
- :code:`widgets` : visualization



In [27]:
import spikeinterface as si  # import core only
import spikeinterface.extractors as se
import spikeinterface.sorters as ss
import spikeinterface.preprocessing as sp
import spikeinterface.comparison as sc
import spikeinterface.widgets as sw
from spikeinterface.exporters import export_to_phy

ModuleNotFoundError: No module named 'spikeinterface.toolkit'

In [8]:
import spikeinterface.core

In [9]:
from probeinterface import get_probe
from probeinterface.plotting import plot_probe, plot_probe_group
from probeinterface import write_prb, read_prb

In [10]:
# import mountainsort5 as ms5

In [11]:
# from tempfile import TemporaryDirectory
# from mountainsort5.util import create_cached_recording

We can also import all submodules at once with this
  this internally import core+extractors+toolkit+sorters+comparison+widgets+exporters

This is useful for notebooks but this is a more heavy import because internally many more dependency
are imported (scipy/sklearn/networkx/matplotlib/h5py...)



In [12]:
import spikeinterface.full as si

In [13]:
# Increase size of plot in jupyter

plt.rcParams["figure.figsize"] = (10,6)

# Part 0: Loading in the Probe

- Reading in the probe information into Spike interface and plotting the probe

In [14]:
probe_object = read_prb("./linear_probe_with_large_spaces.prb")

In [15]:
probe_object.to_dataframe()

,probe_index,x,y,contact_shapes,radius,shank_ids,contact_ids
0,0,0.0,0.0,circle,5.0,,
1,0,5.0,20.0,circle,5.0,,
2,0,-7.0,40.0,circle,5.0,,
3,0,9.0,60.0,circle,5.0,,
4,0,-11.0,80.0,circle,5.0,,
5,0,13.0,100.0,circle,5.0,,
6,0,-15.0,120.0,circle,5.0,,
7,0,17.0,140.0,circle,5.0,,
8,0,-19.0,160.0,circle,5.0,,
9,0,21.0,180.0,circle,5.0,,


In [16]:
probe_object.get_global_contact_ids()

array(['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''],
      dtype='<U64')

In [17]:
probe_object.get_global_device_channel_indices()

array([(0,  0), (0,  1), (0,  2), (0,  3), (0,  4), (0,  5), (0,  6),
       (0,  7), (0,  8), (0,  9), (0, 10), (0, 11), (0, 12), (0, 13),
       (0, 14), (0, 15), (0, 16), (0, 17), (0, 18), (0, 19), (0, 20),
       (0, 21), (0, 22), (0, 23), (0, 24), (0, 25), (0, 26), (0, 27),
       (0, 28), (0, 29), (0, 30), (0, 31)],
      dtype=[('probe_index', '<i8'), ('device_channel_indices', '<i8')])

- Creating a dictionary of all the variables in the probe file

In [18]:
if 'probe_parameters' in locals():
    probe_dict = defaultdict(dict)
    for attribute in dir(probe_parameters):
        # Removing built in attributes
        if not attribute.startswith("__"): 
            probe_dict[attribute] = getattr(probe_parameters, attribute)

In [19]:
if "probe_dict" in locals():
    for key, value in probe_dict.items():
        print("{}: {}".format(key, value))

# Part 1: Importing Data

## Loading in the Electrophysiology Recording

- We are inputting the electrophsiology recording data with probe information. This should have been created in the prevous notebook in a directory created by Spike Interface. If you had already read in your own electrophsiology recording data with probe information with a different way, then follow these instructions.
    - If you want to use a different directory, then you must either:
        - Change `glob.glob({./path/to/with/*/recording_raw})` to the directory that you have the directories created from Spikeinterface. You can use a wildcard if you have multiple folders. You would replace `{./path/to/with/*/recording_raw}` with the path to either the parent directory or the actual directory containing the electrophsiology recording data read into Spikeinterface.
        - Or change `(file_or_folder_or_dict={./path/to/recording_raw})`. You would replace `{./path/to/recording_raw}` with the path to either the parent directory or the actual directory containing the electrophsiology recording data read into Spikeinterface.

In [20]:
recording_filepath_glob = "/scratch/back_up/reward_competition_extention/data/rce_cohort_3/*/*.rec/*merged.rec"

In [21]:
all_recording_files = glob.glob(recording_filepath_glob, recursive=True)

In [22]:
all_recording_files = [file_path for file_path in all_recording_files if "copies" not in file_path]

In [23]:
all_recording_files

['/scratch/back_up/reward_competition_extention/data/rce_cohort_3/discriminate/20240417_150153_comp_discriminate_subj_3-3_and_3-4.rec/20240417_150153_comp_discriminate_subj_3-3_t5b5_merged.rec',
 '/scratch/back_up/reward_competition_extention/data/rce_cohort_3/discriminate/20240417_150153_comp_discriminate_subj_3-3_and_3-4.rec/20240417_150153_comp_discriminate_subj_3-4_t6b6_merged.rec',
 '/scratch/back_up/reward_competition_extention/data/rce_cohort_3/discriminate/20240413_151204_comp_discriminate_subj_3-1_and_3-3.rec/20240413_151204_comp_discriminate_subj_3-1_t5b5_merged.rec',
 '/scratch/back_up/reward_competition_extention/data/rce_cohort_3/discriminate/20240413_151204_comp_discriminate_subj_3-1_and_3-3.rec/20240413_151204_comp_discriminate_subj_3-3_t6b6_merged.rec',
 '/scratch/back_up/reward_competition_extention/data/rce_cohort_3/discriminate/20240414_154115_comp_discriminate_subj_4-2_and_4-4.rec/20240414_154115_comp_discriminate_subj_4-2_t3b3_merged.rec',
 '/scratch/back_up/reward

# Part 2: Sorting

In [24]:
ss.get_default_sorter_params(sorter_name_or_class='kilosort4')

{'batch_size': 60000,
 'nblocks': 1,
 'Th_universal': 9,
 'Th_learned': 8,
 'do_CAR': True,
 'invert_sign': False,
 'nt': 61,
 'artifact_threshold': None,
 'nskip': 25,
 'whitening_range': 32,
 'binning_depth': 5,
 'sig_interp': 20,
 'nt0min': None,
 'dmin': None,
 'dminx': 32,
 'min_template_size': 10,
 'template_sizes': 5,
 'nearest_chans': 10,
 'nearest_templates': 100,
 'templates_from_data': True,
 'n_templates': 6,
 'n_pcs': 6,
 'Th_single_ch': 6,
 'acg_threshold': 0.2,
 'ccg_threshold': 0.25,
 'cluster_downsampling': 20,
 'cluster_pcs': 64,
 'duplicate_spike_bins': 15,
 'do_correction': True,
 'keep_good_only': False,
 'save_extra_kwargs': False,
 'skip_kilosort_preprocessing': False,
 'scaleproc': None,
 'torch_device': 'auto'}

In [25]:
ss.get_sorter_params_description(sorter_name_or_class='kilosort4')

{'batch_size': 'Number of samples included in each batch of data.',
 'nblocks': 'Number of non-overlapping blocks for drift correction (additional nblocks-1 blocks are created in the overlaps). Default value: 1.',
 'Th_universal': 'Spike detection threshold for universal templates. Th(1) in previous versions of Kilosort. Default value: 9.',
 'Th_learned': 'Spike detection threshold for learned templates. Th(2) in previous versions of Kilosort. Default value: 8.',
 'do_CAR': 'Whether to perform common average reference. Default value: True.',
 'invert_sign': 'Invert the sign of the data. Default value: False.',
 'nt': 'Number of samples per waveform. Also size of symmetric padding for filtering. Default value: 61.',
 'artifact_threshold': 'If a batch contains absolute values above this number, it will be zeroed out under the assumption that a recording artifact is present. By default, the threshold is infinite (so that no zeroing occurs). Default value: None.',
 'nskip': 'Batch stride f

In [26]:
successful_files = [] 
failed_files = []

for recording_file in all_recording_files:
    print(recording_file)
    
    recording_basename = os.path.basename(recording_file)
    recording_output_directory = "/scratch/back_up/reward_competition_extention/proc/spike_sorting/old/kilosort4/independent_channel/{}".format(recording_basename)
    os.makedirs(recording_output_directory, exist_ok=True)

    print("Output directory: {}".format(recording_output_directory))
    
    try:
        with open('successful_files.txt', "r") as myfile:
            if recording_basename in myfile.read():
                warnings.warn("""Directory already exists for: {}.
                              Either continue on if you are satisfied with the previous run 
                              or delete the directory and run this cell again""".format(recording_basename))
                continue
    except:
        pass
    
    try:
        trodes_recording = se.read_spikegadgets(recording_file, stream_id="trodes")       
        trodes_recording = trodes_recording.set_probes(probe_object)
        
        clipped_recording: si.BaseRecording = sp.resample(trodes_recording, resample_rate=20000, dtype=np.float32)
        
        clipped_recording = st.preprocessing.remove_bad_channels(clipped_recording, bad_channel_ids=None, bad_threshold=2,
                                                                 seconds=2)

        child_spikesorting_output_directory = os.path.join(recording_output_directory,"ss_output")
               
        start = time.time()
        # Make sure the recording is preprocessed appropriately
        # lazy preprocessing
        print("Running bandpass filter")
        recording_filtered = sp.bandpass_filter(trodes_recording, freq_min=300, freq_max=6000, dtype=np.float32)

        print("Running whitening")
        recording_preprocessed: si.BaseRecording = sp.whiten(recording_filtered, dtype=np.float32)
                
        print("Spike sorting")
        spike_sorted_object = ss.run_sorter(sorter_name="kilosort4", 
                                            recording=clipped_recording,
                                            skip_kilosort_preprocessing=False,
                                            do_CAR=False,
                                            do_correction=False,
                                            nearest_chans=1,
                                            nt= 41,
                                            Th_universal = 9 + 3,
                                            Th_learned = 8 + 3,
                                            nearest_templates=1,
                                            output_folder=child_spikesorting_output_directory
                                            )
                
        shutil.rmtree(child_spikesorting_output_directory, ignore_errors=True)
        spike_sorted_object.save(folder=child_spikesorting_output_directory)

        sw.plot_rasters(spike_sorted_object)
        plt.title(recording_basename)
        plt.ylabel("Unit IDs")
        
        plt.savefig(os.path.join(recording_output_directory, "{}_raster_plot.png".format(recording_basename)))
        plt.close()
        
        print("Exporting waveforms")
        waveform_output_directory = os.path.join(recording_output_directory, "waveforms")
        we_spike_sorted = si.extract_waveforms(
            recording=recording_preprocessed, 
            sorting=spike_sorted_object, 
            folder=waveform_output_directory,
            ms_before=1, 
            ms_after=1, 
            progress_bar=True,
            n_jobs=-1, 
            total_memory="8G", 
            overwrite=True,
            max_spikes_per_unit=2000,
            sparse=False)
        
        print("we_spike_sorted is sparse: {}".format(we_spike_sorted.is_sparse()))
        
        print("Saving to phy")
        phy_output_directory = os.path.join(recording_output_directory, "phy")
        export_to_phy(we_spike_sorted, 
                        phy_output_directory,
                        compute_pc_features=True, 
                        compute_amplitudes=True, 
                        remove_if_exists=True)
        
        # edit the params.py file os that it contains the correct realtive path
        params_path = os.path.join(phy_output_directory, "params.py")
        with open(params_path, 'r') as file:
            lines = file.readlines()
        lines[0] = "dat_path = r'./recording.dat'\n"
        with open(params_path, 'w') as file:
            file.writelines(lines)
        
        successful_files.append(recording_file)
        with open('successful_files.txt', "a+") as fd:
            fd.write(f'\n{recording_basename}')                     

    except Exception as e: 
        print(e)
        failed_files.append(recording_file)

/scratch/back_up/reward_competition_extention/data/rce_cohort_3/discriminate/20240417_150153_comp_discriminate_subj_3-3_and_3-4.rec/20240417_150153_comp_discriminate_subj_3-3_t5b5_merged.rec
Output directory: /scratch/back_up/reward_competition_extention/proc/spike_sorting/old/kilosort4/independent_channel/20240417_150153_comp_discriminate_subj_3-3_t5b5_merged.rec
Running bandpass filter
Running whitening
Spike sorting
Loading recording with SpikeInterface...
number of samples: 72856289
number of channels: 32
numbef of segments: 1
sampling rate: 20000.0
dtype: float32
Skipping common average reference.
Skipping kilosort preprocessing.
Skipping drift correction.

computing drift
nblocks = 0, skipping drift correction
drift computed in  0.00s; total  0.00s

Extracting spikes using templates
Re-computing universal templates from data.


100%|██████████| 1215/1215 [03:07<00:00,  6.48it/s]


12145494 spikes extracted in  194.91s; total  194.92s

First clustering


 19%|█▉        | 3/16 [00:55<04:18, 19.88s/it]/home/pc_lab/miniforge3/envs/kilosort_4_0_5/lib/python3.9/site-packages/scipy/sparse/_index.py:151: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
100%|██████████| 16/16 [05:00<00:00, 18.78s/it]
/home/pc_lab/miniforge3/envs/kilosort_4_0_5/lib/python3.9/site-packages/kilosort/template_matching.py:71: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at /opt/conda/conda-bld/pytorch_1712608885084/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  X = conv1d(X.unsqueeze(1), ops['wTEMP'].unsqueeze(1), padding=ops['nt']//2)


229 clusters found, in  301.39s; total  496.30s

Extracting spikes using cluster waveforms


...

Run:
phy template-gui  /scratch/back_up/reward_competition_extention/proc/spike_sorting/old/kilosort4/independent_channel/20240417_150153_comp_discriminate_subj_3-3_t5b5_merged.rec/phy/params.py
/scratch/back_up/reward_competition_extention/data/rce_cohort_3/discriminate/20240417_150153_comp_discriminate_subj_3-3_and_3-4.rec/20240417_150153_comp_discriminate_subj_3-4_t6b6_merged.rec
Output directory: /scratch/back_up/reward_competition_extention/proc/spike_sorting/old/kilosort4/independent_channel/20240417_150153_comp_discriminate_subj_3-4_t6b6_merged.rec
Running bandpass filter
Running whitening
Spike sorting
Loading recording with SpikeInterface...
number of samples: 72861495
number of channels: 32
numbef of segments: 1
sampling rate: 20000.0
dtype: float32
Skipping common average reference.
Skipping kilosort preprocessing.
Skipping drift correction.

computing drift
nblocks = 0, skipping drift correction
drift computed in  0.00s; total  0.01s

Extracting spikes using templates
Re-co

100%|██████████| 1215/1215 [03:19<00:00,  6.10it/s]


11666894 spikes extracted in  205.87s; total  205.88s

First clustering


 19%|█▉        | 3/16 [00:27<01:52,  8.66s/it]/home/pc_lab/miniforge3/envs/kilosort_4_0_5/lib/python3.9/site-packages/scipy/sparse/_index.py:151: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
 69%|██████▉   | 11/16 [04:00<01:49, 21.89s/it]


KeyboardInterrupt: 

In [ ]:
# successful_files = [] 
# failed_files = []

# for recording_file in all_recording_files:
#     print(recording_file)
    
#     recording_basename = os.path.basename(recording_file)
#     recording_output_directory = "/scratch/back_up/reward_competition_extention/proc/spike_sorting/{}".format(recording_basename)
#     os.makedirs(recording_output_directory, exist_ok=True)

#     print("Output directory: {}".format(recording_output_directory))
    
#     try:
#         with open('successful_files.txt', "r") as myfile:
#             if recording_basename in myfile.read():
#                 warnings.warn("""Directory already exists for: {}.
#                               Either continue on if you are satisfied with the previous run 
#                               or delete the directory and run this cell again""".format(recording_basename))
#                 continue
#     except:
#         pass
    
#     try:
#         trodes_recording = se.read_spikegadgets(recording_file, stream_id="trodes")       
#         trodes_recording = trodes_recording.set_probes(probe_object)

#         child_spikesorting_output_directory = os.path.join(recording_output_directory,"ss_output")
               
#         start = time.time()
#         # Make sure the recording is preprocessed appropriately
#         # lazy preprocessing
#         print("Running bandpass filter")
#         recording_filtered = sp.bandpass_filter(trodes_recording, freq_min=300, freq_max=6000, dtype=np.float32)

#         print("Running whitening")
#         recording_preprocessed: si.BaseRecording = sp.whiten(recording_filtered, dtype=np.float32)
                
#         print("Spike sorting")
#         spike_sorted_object = ss.run_sorter(sorter_name="kilosort4", 
#                                             recording=recording_preprocessed,
#                                             skip_kilosort_preprocessing=True,
#                                             do_CAR=False,
#                                             do_correction=False,
#                                             output_folder=child_spikesorting_output_directory
#                                             )
                
#         shutil.rmtree(child_spikesorting_output_directory, ignore_errors=True)
#         spike_sorted_object.save(folder=child_spikesorting_output_directory)

#         sw.plot_rasters(spike_sorted_object)
#         plt.title(recording_basename)
#         plt.ylabel("Unit IDs")
        
#         plt.savefig(os.path.join(recording_output_directory, "{}_raster_plot.png".format(recording_basename)))
#         plt.close()
        
#         print("Exporting waveforms")
#         waveform_output_directory = os.path.join(recording_output_directory, "waveforms")
#         we_spike_sorted = si.extract_waveforms(
#             recording=recording_preprocessed, 
#             sorting=spike_sorted_object, 
#             folder=waveform_output_directory,
#             ms_before=1, 
#             ms_after=1, 
#             progress_bar=True,
#             n_jobs=-1, 
#             total_memory="8G", 
#             overwrite=True,
#             max_spikes_per_unit=2000,
#             sparse=False)
        
#         print("we_spike_sorted is sparse: {}".format(we_spike_sorted.is_sparse()))
        
#         print("Saving to phy")
#         phy_output_directory = os.path.join(recording_output_directory, "phy")
#         export_to_phy(we_spike_sorted, 
#                         phy_output_directory,
#                         compute_pc_features=True, 
#                         compute_amplitudes=True, 
#                         remove_if_exists=True)
        
#         # edit the params.py file os that it contains the correct realtive path
#         params_path = os.path.join(phy_output_directory, "params.py")
#         with open(params_path, 'r') as file:
#             lines = file.readlines()
#         lines[0] = "dat_path = r'./recording.dat'\n"
#         with open(params_path, 'w') as file:
#             file.writelines(lines)
        
#         successful_files.append(recording_file)
#         with open('successful_files.txt', "a+") as fd:
#             fd.write(f'\n{recording_basename}')                     

#     except Exception as e: 
#         print(e)
#         failed_files.append(recording_file)


In [ ]:
# successful_files = [] 
# failed_files = []

# for recording_file in all_recording_files:
#     print(recording_file)
    
#     recording_basename = os.path.basename(recording_file)
#     recording_output_directory = "/scratch/back_up/reward_competition_extention/proc/spike_sorting/{}".format(recording_basename)
#     os.makedirs(recording_output_directory, exist_ok=True)

#     print("Output directory: {}".format(recording_output_directory))
    
#     try:
#         with open('successful_files.txt', "r") as myfile:
#             if recording_basename in myfile.read():
#                 warnings.warn("""Directory already exists for: {}.
#                               Either continue on if you are satisfied with the previous run 
#                               or delete the directory and run this cell again""".format(recording_basename))
#                 continue
#     except:
#         pass
    
#     try:
#         trodes_recording = se.read_spikegadgets(recording_file, stream_id="trodes")       
#         trodes_recording = trodes_recording.set_probes(probe_object)

#         child_spikesorting_output_directory = os.path.join(recording_output_directory,"ss_output")
               
#         start = time.time()
#         # Make sure the recording is preprocessed appropriately
#         # lazy preprocessing
#         print("Running bandpass filter")
#         recording_filtered = sp.bandpass_filter(trodes_recording, freq_min=300, freq_max=6000, dtype=np.float32)

#         print("Running whitening")
#         recording_preprocessed: si.BaseRecording = sp.whiten(recording_filtered, dtype=np.float32)
        
#         with TemporaryDirectory() as tmpdir:
#             # cache the recording to a temporary directory for efficient reading
#             print("Caching the recording")
#             recording_cached = create_cached_recording(recording_preprocessed, folder=tmpdir)
#             recording_cached = recording_cached.set_probes(probe_object)
#             recording_cached.annotate(is_filtered=True)

#             print("Spike sorting")
#             spike_sorted_object = ms5.sorting_scheme2(
#                 recording=recording_cached,
#                 sorting_parameters=ms5.Scheme2SortingParameters(
#                     detect_sign=0,
#                     phase1_detect_channel_radius=700,
#                     detect_channel_radius=700,
#                 # other parameters...
#                 )
#             )
            
#             assert isinstance(spike_sorted_object, si.BaseSorting)
            
#             shutil.rmtree(child_spikesorting_output_directory, ignore_errors=True)
#             spike_sorted_object.save(folder=child_spikesorting_output_directory)

#             sw.plot_rasters(spike_sorted_object)
#             plt.title(recording_basename)
#             plt.ylabel("Unit IDs")
            
#             plt.savefig(os.path.join(recording_output_directory, "{}_raster_plot.png".format(recording_basename)))
#             plt.close()
            
#             print("Exporting waveforms")
#             waveform_output_directory = os.path.join(recording_output_directory, "waveforms")
#             we_spike_sorted = si.extract_waveforms(
#                 recording=recording_cached, 
#                 sorting=spike_sorted_object, 
#                 folder=waveform_output_directory,
#                 ms_before=1, 
#                 ms_after=1, 
#                 progress_bar=True,
#                 n_jobs=-1, 
#                 total_memory="8G", 
#                 overwrite=True,
#                 max_spikes_per_unit=2000,
#                 sparse=False)
            
#             print("we_spike_sorted is sparse: {}".format(we_spike_sorted.is_sparse()))
            
#             print("Saving to phy")
#             phy_output_directory = os.path.join(recording_output_directory, "phy")
#             export_to_phy(we_spike_sorted, 
#                             phy_output_directory,
#                             compute_pc_features=True, 
#                             compute_amplitudes=True, 
#                             remove_if_exists=True)
            
#             # edit the params.py file os that it contains the correct realtive path
#             params_path = os.path.join(phy_output_directory, "params.py")
#             with open(params_path, 'r') as file:
#                 lines = file.readlines()
#             lines[0] = "dat_path = r'./recording.dat'\n"
#             with open(params_path, 'w') as file:
#                 file.writelines(lines)
            
#             successful_files.append(recording_file)
#             with open('successful_files.txt', "a+") as fd:
#                 fd.write(f'\n{recording_basename}')                     

#     except Exception as e: 
#         print(e)
#         failed_files.append(recording_file)


In [ ]:
raise ValueError()